In [1]:
from web3 import Web3, IPCProvider
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
w3 = Web3(IPCProvider('/shakespeare/sandipan/Ethereum/.ethereum/geth.ipc'))
w3.eth.getBlock('latest')

AttributeDict({'difficulty': 2400374661886632,
 'extraData': HexBytes('0x4554482e45544846414e532e4f52472d3136313946374532'),
 'gasLimit': 7992111,
 'gasUsed': 7975053,
 'hash': HexBytes('0xf87f524c2e8542d6b70661c2067c628a70b6eaca7c173a7baaaf74a2a72a962f'),
 'logsBloom': HexBytes('0x40a6616024418010024da00004900029008081210015020080aa6400023821280000311020042608403009860240741341824486940030109c184b102c0042109000b16100210501403077493120c000060100c8180400800401c0040ec00008100098600a5240804a41c1e69002080002400213c80441780200851140a0c20c3622c10000c3004284180006000350308109088102a0031c200220002000214040408248020b00004e1001f25002840390800d0900108284500821402010a490144a0c06281d8220005e009802008c880c02209583c908026001100008802191301000c00022042051e184818002a004500442510098c4090a001c002ac11289'),
 'miner': '0x5A0b54D5dc17e0AadC383d2db43B0a0D3E029c4c',
 'mixHash': HexBytes('0x50ada62ae9af3fa9745e3f07596b1c26cea0dde969b55ec8847b86ad7fda7d1f'),
 'nonce': HexBytes('0xbbc40b180ad30134'),
 'number': 

In [3]:
#Read the levels into data frame
genesis = pd.read_csv('genesisLevel.csv')
genesis1 = pd.read_csv('genesisLevel_1.csv')
genesis = genesis.append(genesis1, ignore_index=True)

In [4]:
from multiprocessing import Process,Manager
manager = Manager()
genesisLevel = manager.dict() #stores the cumulative Balance of a level
genesisBalance = manager.dict() #stores the balance per account
curr_level = 3

In [5]:
def getBalance(_addrList, _lvl ):
    for addr in _addrList:
        addCSA = Web3.toChecksumAddress(addr)
        try:
            balance = w3.eth.getBalance(addCSA)
        except:
            retries.append(addCSA)
            balance = 0 #in case balance was not set previously
        genesisLevel[_lvl] = genesisLevel.get(_lvl,0) + balance #look up the current balance, if no entry return 0
        genesisBalance[addr] = balance
    print('The length of the retries is %i' %len(retries))

In [ ]:
#Compute the cumulative and individual balances for each level
maxLevel = max(genesis.itertuples(), key=lambda x:x[2] )[2]
retries = []


while curr_level <= maxLevel:
    print('looking into level: ',curr_level)
    jobs = []
    addresses = [addr for (i,addr,lvl) in genesis.itertuples() if lvl == curr_level]
    print('obtained addresses for the next level')
    print('addresses obtained: ',len(addresses))
    if len(addresses)>100:
        job_cnt = 5
    else:
        job_cnt = 1
   
    partition_size = int(len(addresses)/job_cnt) + 1
    start_i = 0
    end_i = start_i + partition_size
    for i in range(job_cnt):
        if i==job_cnt-1:
            p = Process(target=getBalance, args=(addresses[start_i:],curr_level))
        else:
            p = Process(target=getBalance, args=(addresses[start_i:end_i],curr_level))
        start_i = end_i
        end_i = start_i + partition_size
        jobs.append(p)
        p.start()

    for proc in jobs:
        proc.join()
    
    
    print('filled the dictionary level values')
    ft = open('balancesLvl.csv','a')
    for addr in addresses:
        currentAddr = addr
        try:
            ft.write(currentAddr+','+str(genesisBalance[currentAddr])+','+str(curr_level))
        except:
            ft.write('Error with the Adress: '+ currentAddr)
        ft.write('\n')
    ft.close()
    
    curr_level+=1

looking into level:  3
obtained addresses for the next level
addresses obtained:  3563436


In [1]:
plt.loglog(genesisLevel.values())
plt.show()

NameError: name 'plt' is not defined

In [ ]:
#compute the gini coefficient of the levels
def giniCoefficient(df_balance): # balance is pandas table....
    balance = df_balance.Balance.sort_values()
    b = balance.values # convert to a numpy array
    n = b.size
    num = sum(list(map(lambda x: (n-x[0])*x[1], enumerate(b))))
    den = b.sum()
    return (n+1-2*(num/den))/n

In [ ]:
address = np.append(df_b.From.unique(),df_b.To.unique())

In [ ]:
blockNum = 100
for i in range(len(address)):
    if address[i]!='ethereum':
        account = Web3.toChecksumAddress(address[i])
        balance = w3.eth.getBalance(account,blockNum)
        print(address[i],balance)

In [ ]:
def fillBalanceTable(blockNum,fname):
    df_b = df[df['Block']==blockNum][['From','To']]
    address = np.append(df_b.From.unique(),df_b.To.unique())
    #with open(fname,'a') as ft:
    for i in range(len(address)):
        if address[i]!='ethereum' and address[i]!='0':
            account = Web3.toChecksumAddress(address[i])
            balance = w3.eth.getBalance(account,blockNum)
            ft = open(fname,'a')
            ft.write(account+','+str(balance)+','+str(blockNum))
            ft.write('\n')
            ft.close()

In [ ]:
fillBalanceTable(10000)

In [ ]:
df[df.Block==10000]

In [ ]:
fs = open('blockAccountBalance1.csv','w')
fs.write('Account,Balance,Block\n')
fs.close()

In [ ]:
for i in range(1,1000):
    fillBalanceTable(i,'blockAccountBalance.csv')

In [ ]:
df_1 = pd.read_csv('blockAccountBalance.csv')

In [ ]:
df_1.head()

In [ ]:
df_1.Block.max()

In [ ]:
for i in range(1000,5001):
    fillBalanceTable(i,'blockAccountBalance.csv')

In [ ]:
def createBalanceCSV(start,end,fname):
    for i in range(start,end+1):
        fillBalanceTable(i,fname)
        print('finished for block: ',i)

In [ ]:
from multiprocessing import Process

In [ ]:
jobs = []

In [ ]:
jobs.append(Process(target=createBalanceCSV, args=(5305,7500,"blockAccountBalance.csv")))
jobs.append(Process(target=createBalanceCSV, args=(7501,10000,"blockAccountBalance.csv")))
jobs.append(Process(target=createBalanceCSV, args=(10001,12500,"blockAccountBalance.csv")))
jobs.append(Process(target=createBalanceCSV, args=(12501,15000,"blockAccountBalance.csv")))

In [ ]:
for j in jobs:
    j.start()
    for j in jobs:
        j.join()

In [ ]:
df[df.Block==5305]

In [ ]:
import numpy as np
def balanceAccountBlock(blockNum, df): # retrieve balances of all accounts given a block number
    df_b = df[df['Block']<=blockNum][['From','To']]
    address = np.append(df_b.From.unique(),df_b.To.unique())
    address = list(set(address))
    print("addresses to look into: ",len(address))
    with open('AccountBalanace'+str(blockNum)+'.csv','w') as ft:
        ft.write('Account,Balance,Block')
        ft.write('\n')
        for add in address:
            if add!='ethereum' and add!='0':
                try:
                    account = Web3.toChecksumAddress(add)
                    balance = w3.eth.getBalance(account,blockNum)
                    ft.write(add+','+str(balance)+','+str(blockNum))
                    ft.write('\n')
                except:
                    print(add)
    

In [ ]:
balanceAccountBlock(100000,df)

In [ ]:
account = Web3.toChecksumAddress('c47d9e6c75d388aeb92574429dad43a148f2f3ea')

In [ ]:
x = Web3.fromWei(25000000000000,'ether')

In [ ]:
df_balance.Balance = df_balance.Balance.apply(lambda x: float(x))

In [ ]:
df[df.To=='6.12416344436736E+039']

In [ ]:
balance = df_balance.Balance.sort_values()

In [ ]:
G_coeff = 0.0
balance.iloc[0]

In [ ]:
n = balance.count()

In [ ]:
n

In [ ]:
balance.iloc[12623]

In [ ]:
b = balance.values

In [ ]:
b

In [ ]:
b.size

In [ ]:
b.cumsum()

In [ ]:
a = np.array([1,2,3,4,5,6])

In [ ]:
b.sum()

In [ ]:
A = [1,2,3,4,5]
n = len(A)+1
c = map(lambda(i,x):i*x, enumerate(A))

In [ ]:
a = [0, 0, 0, 0, 1]
a.sort()
n = len(a)
num = sum(list(map(lambda x: (n-x[0])*x[1], enumerate(a))))
den = sum(a)

In [ ]:
(n+1 - 2*(num/den))/n

In [ ]:
n = b.size
answer = list(map(lambda x: (n-x[0])*x[1], enumerate(b)))

In [ ]:
a = np.array(a)
for i,j in enumerate(a):
    print(i,j)

In [ ]:
(n+1-2*sum(answer)/b.sum())/n

In [ ]:
giniCoefficient(df_balance)

In [ ]:
df.columns

In [ ]:
x = df.groupby('From').count()['To']
y = df.groupby('To').count()['From']

In [ ]:
df[df['From']=='0004b15be8cb573a7be023e203e268dbcffd302b']

In [ ]:
z = pd.concat([x,y])

In [ ]:
df[df['To']=='0004b15be8cb573a7be023e203e268dbcffd302b']

In [ ]:
Web3.isAddress('fee4deb01301a2633e0046305a31fca8d04f416c')

In [ ]:
add = Web3.toChecksumAddress('ffcd9cae6d9f54fd8ae05296f839892677ff6eaa')

In [ ]:
len(w3.eth.getCode(add))

In [ ]:
def isAccount(address):
    cs_add = Web3.toChecksumAddress(address)
    if len(w3.eth.getCode(cs_add))==0:
        return True
    return False